In [ ]:
import numpy as np
import pandas as pd

In [3]:
prompts_train = pd.read_csv("/content/prompts_train.csv")
prompts_test = pd.read_csv("/content/prompts_test.csv")

summaries_train = pd.read_csv("/content/summaries_train.csv")
summaries_test = pd.read_csv("/content/summaries_test.csv")

sample_submission = pd.read_csv("/content/sample_submission.csv")

prompts_train

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


In [5]:
train = summaries_train.merge(prompts_train, how="left", on="prompt_id")
test = summaries_test.merge(prompts_test, how="left", on="prompt_id")

train

,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
...,...,...,...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...,0.205683,0.380538,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...,-0.308448,0.048171,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...,-1.408180,-0.493603,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...,-0.393310,0.627128,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...


In [6]:
class CFG:
    model_name="/kaggle/input/debertav3base"
    learning_rate=1.5e-5
    weight_decay=0.02
    hidden_dropout_prob=0.005
    attention_probs_dropout_prob=0.005
    num_train_epochs=3
    n_splits=4
    batch_size=12
    random_seed=42
    save_steps=100
    max_length=512

In [ ]:
import warnings
import logging
import os
import shutil
import json
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from datasets import Dataset,load_dataset, load_from_disk
from transformers import TrainingArguments, Trainer
from datasets import load_metric, disable_progress_bar
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
import torch

warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
disable_progress_bar()


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }

def compt_score(content_true, content_pred, wording_true, wording_pred):
    content_score = mean_squared_error(content_true, content_pred)**(1/2)
    wording_score = mean_squared_error(wording_true, wording_pred)**(1/2)

    return (content_score + wording_score)/2

def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(seed=42)

In [ ]:
def train_n_infer(train,
                  val,
                  model_name="roberta-base",
                  batch_size=4,
                  learning_rate=5e-5,
                  weight_decay=0.01,
                  hidden_dropout_prob=0.0,
                  attention_probs_dropout_prob=0.0,
                  num_train_epochs=2,
                  save_steps=100,
                  random_seed=42,
                  max_length=512,
                  model_dir="bert"):
    train_content = train[["text", "content", "wording"]]
    val_content = val[["text", "content", "wording"]]
    test_ = test[["text"]]

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model_config = AutoConfig.from_pretrained(model_name)
    model_config.update({
        "hidden_dropout_prob": hidden_dropout_prob, # 0.0
        "attention_probs_dropout_prob": attention_probs_dropout_prob,
        "num_labels": 2,
        "problem_type": "regression",
    })
    seed_everything(seed=42)
    model_content = AutoModelForSequenceClassification.from_pretrained(model_name, config=model_config)

    train_dataset_content = Dataset.from_pandas(train_content, preserve_index=False)
    val_dataset_content = Dataset.from_pandas(val_content, preserve_index=False)

    def tokenize_function(examples):
        labels = [examples["content"], examples["wording"]]
        tokenized = tokenizer(examples["text"],
                         padding=False,
                         truncation=True,
                         max_length=max_length)
        return {
            **tokenized,
            "labels": labels,
        }

    def tokenize_function_test(examples):
        tokenized = tokenizer(examples["text"],
                         padding=False,
                         truncation=True,
                         max_length=max_length)
        return tokenized

    train_tokenized_datasets_content = train_dataset_content.map(tokenize_function, batched=False)
    val_tokenized_datasets_content = val_dataset_content.map(tokenize_function, batched=False)

    test_dataset = Dataset.from_pandas(test_, preserve_index=False)
    test_tokenized_dataset = test_dataset.map(tokenize_function_test, batched=False)

    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer
    )
    model_dir = model_dir
    training_args = TrainingArguments(
        output_dir=model_dir,
        load_best_model_at_end=True,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=8,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        report_to='none',
        greater_is_better=False,
        save_strategy="steps",
        evaluation_strategy="steps",
        eval_steps=save_steps,
        save_steps=save_steps,
        metric_for_best_model="mcrmse",
        save_total_limit=1
    )

    trainer_content = Trainer(
        model=model_content,
        args=training_args,
        train_dataset=train_tokenized_datasets_content,
        eval_dataset=val_tokenized_datasets_content,
        tokenizer=tokenizer,
        compute_metrics=compute_mcrmse,#compute_metrics,
        data_collator=data_collator
    )

    trainer_content.train()

    best_check = os.listdir(model_dir)[0]
    model_content = AutoModelForSequenceClassification.from_pretrained(f"{model_dir}/{best_check}")
    model_content.eval()

    test_args = TrainingArguments(
        output_dir=model_dir,
        do_train = False,
        do_predict = True,
        per_device_eval_batch_size = 4,
        dataloader_drop_last = False,
    )

    # init trainer
    infer_content = Trainer(
                  model = model_content,
                  tokenizer=tokenizer,
                  data_collator=data_collator,
                  args = test_args)

    val_results_content = infer_content.predict(val_tokenized_datasets_content)[0]
    test_results_content = infer_content.predict(test_tokenized_dataset)[0]

    model_content.save_pretrained(model_dir)
    tokenizer.save_pretrained(model_dir)
    shutil.rmtree(f"{model_dir}/{best_check}")

    return val_results_content, test_results_content

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import KFold, GroupKFold


def get_oof_pred_n_test(train,
                        model_name="/kaggle/input/berttiny",
                        n_splits=3,
                        batch_size=4,
                        learning_rate=5e-5,
                        hidden_dropout_prob=0.0,
                        attention_probs_dropout_prob=0.0,
                        weight_decay=0.01,
                        num_train_epochs=2,
                        random_seed=42,
                        save_steps=100,
                        max_length=512
                       ):
#     kfl = KFold(n_splits=n_splits, random_state=random_seed, shuffle=True)
    kfl = GroupKFold(n_splits=n_splits)
    oof_content = np.zeros((len(train), 2))
    test_pred_content = np.zeros((len(test), 2))
    model_name_ = model_name.split("/")[-1]
    if os.path.exists(model_name_):
        shutil.rmtree(model_name_)
    os.mkdir(model_name_)
    for i, (train_indx, val_indx) in enumerate(kfl.split(train, groups=train["prompt_id"])):
        print(f"fold {i}:")
        train_ = train.iloc[train_indx]
        val_ = train.iloc[val_indx]

        val_res_content, test_res_content  = train_n_infer(train_,
                                                           val_,
                                                           model_name=model_name,
                                                           batch_size=batch_size,
                                                           learning_rate=learning_rate,
                                                           hidden_dropout_prob=hidden_dropout_prob,
                                                           attention_probs_dropout_prob=attention_probs_dropout_prob,
                                                           weight_decay=weight_decay,
                                                           num_train_epochs=num_train_epochs,
                                                           save_steps=save_steps,
                                                           max_length=max_length,
                                                           random_seed=random_seed,
                                                           model_dir=f"{model_name_}/fold_{i}"
                                                           )

        oof_content[val_indx] = val_res_content
        test_pred_content += test_res_content/n_splits
    oof_train = pd.DataFrame(oof_content, columns=[f"content_pred_{model_name_}", f"wording_pred_{model_name_}"])
    test_pred = pd.DataFrame(test_pred_content, columns=[f"content_pred_{model_name_}", f"wording_pred_{model_name_}"])

    cv_metric = compute_mcrmse((oof_train.values, train[["content", "wording"]]))
    print(f"cv mcrmse: {cv_metric}")
    with open(f"{model_name_}/cv_metric.json", "w") as outfile:
        json.dump(cv_metric, outfile)
    oof_train.to_csv(f"{model_name_}/oof_train.csv", index=False)

    return oof_train, test_pred

In [ ]:
oof_train, test_pred  = get_oof_pred_n_test(train,
                                            model_name=CFG.model_name,
                                            learning_rate=CFG.learning_rate,
                                            hidden_dropout_prob=CFG.hidden_dropout_prob,
                                            attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
                                            weight_decay=CFG.weight_decay,
                                            num_train_epochs=CFG.num_train_epochs,
                                            n_splits=CFG.n_splits,
                                            batch_size=CFG.batch_size,
                                            random_seed=CFG.random_seed,
                                            save_steps=CFG.save_steps,
                                            max_length=CFG.max_length
                                           )

fold 0:


Step,Training Loss,Validation Loss,Content Rmse,Wording Rmse,Mcrmse
100,No log,0.378743,0.535576,0.686036,0.610806
200,No log,0.349388,0.545466,0.633437,0.589452
300,No log,0.251441,0.416284,0.574099,0.495191
400,No log,0.338694,0.587546,0.576349,0.581947
500,0.351300,0.271521,0.488456,0.551773,0.520114
600,0.351300,0.261738,0.409194,0.596688,0.502941
700,0.351300,0.264112,0.474894,0.550182,0.512538
800,0.351300,0.304386,0.555460,0.547937,0.551699
900,0.351300,0.277662,0.514124,0.539445,0.526784
1000,0.195100,0.279434,0.501657,0.554264,0.527960


fold 1:


Step,Training Loss,Validation Loss,Content Rmse,Wording Rmse,Mcrmse
100,No log,0.476645,0.527715,0.821466,0.674590
200,No log,0.426156,0.535086,0.752326,0.643706
300,No log,0.441401,0.515353,0.785629,0.650491
400,No log,0.453456,0.522105,0.796442,0.659273
500,0.325700,0.466135,0.509714,0.820038,0.664876
600,0.325700,0.415590,0.498538,0.763308,0.630923
700,0.325700,0.459316,0.517664,0.806633,0.662148
800,0.325700,0.422963,0.503096,0.769948,0.636522
900,0.325700,0.460277,0.498040,0.820067,0.659054
1000,0.180300,0.455963,0.500785,0.813105,0.656945


fold 2:


Step,Training Loss,Validation Loss,Content Rmse,Wording Rmse,Mcrmse
100,No log,0.393725,0.539486,0.704560,0.622023
200,No log,0.314690,0.468480,0.640239,0.554360
300,No log,0.289074,0.464181,0.602233,0.533207
400,No log,0.302352,0.523317,0.575189,0.549253
500,0.384000,0.276903,0.471852,0.575466,0.523659
600,0.384000,0.318116,0.482048,0.635501,0.558774
700,0.384000,0.331455,0.577496,0.573941,0.575718
800,0.384000,0.276521,0.503438,0.547350,0.525394
900,0.384000,0.283524,0.518035,0.546523,0.532279
1000,0.199900,0.298029,0.534326,0.557272,0.545799


fold 3:


Step,Training Loss,Validation Loss,Content Rmse,Wording Rmse,Mcrmse
100,No log,0.878322,0.694638,1.128770,0.911704
200,No log,0.701924,0.735659,0.928791,0.832225
300,No log,0.622536,0.600440,0.940502,0.770471
400,No log,0.496143,0.569302,0.817424,0.693363
500,0.323000,0.583241,0.709702,0.814128,0.761915
600,0.323000,0.577058,0.760956,0.758329,0.759642
700,0.323000,0.576160,0.701188,0.812807,0.756998
800,0.323000,0.482311,0.640273,0.744764,0.692519
900,0.323000,0.559593,0.698408,0.794614,0.746511
1000,0.189700,0.524733,0.742553,0.705749,0.724151


cv mcrmse: {'content_rmse': 0.49454136395385934, 'wording_rmse': 0.6600226796503624, 'mcrmse': 0.5772820218021109}


In [ ]:
sample_submission["content"] = test_pred.values[:, 0]
sample_submission["wording"] = test_pred.values[:, 1]

sample_submission

,student_id,content,wording
0,000000ffffff,-1.309139,-1.040057
1,111111eeeeee,-1.308928,-1.051735
2,222222cccccc,-1.302874,-1.041919
3,333333dddddd,-1.315247,-1.053631


In [ ]:
sample_submission.to_csv("submission.csv", index=False)